In this question our group focus on why bus services not accessable in some neighborhoods. According to question one, we found that some places with no bus services accessable. To help planning the bus routes, we may want to find out why these places have no bus services, by different factors: population, Road segments, income. 

In [5]:
import numpy as np
import pandas as pd
from pandas import value_counts
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
from shapely.geometry import shape, GeometryCollection, Point

In [4]:
road_segments = pd.read_csv('road-segments.csv')
road_segments.head()

,ROAD_ELEMENT_ID,STREET_BODY_NAME,STREET_TYPE_SUFFIX,DIRECTION_SUFFIX,STREET_CP,FROM_STREET,TO_STREET,PLOW_ROUTE,F_ADDR_L,T_ADDR_L,...,TRAFFIC_FLOW,STATUS,MUNICIPALITY,RIGHT_MUNICIPALITY,LEFT_MUNICIPALITY,ROAD_LEVEL,STREET_TYPE_SHORT,MAP_LABEL,GeoJSON,geo_point_2d
0,2046,CATARAQUI WOODS,DR,NaN,CATARAQUI WOODS DR,CLYDE CRT,GARDINERS RD,31A,NaN,NaN,...,BOTH,ACTIVE,KINGSTON,KINGSTON,KINGSTON,Ground Level,DR,Cataraqui Woods Dr,"{""coordinates"": [[-76.56382462128522, 44.26797...","44.26720550912333,-76.56631192730329"
1,1345,VICTORIA,ST,NaN,VICTORIA ST,CONCESSION ST,NORTHERLY END,22B,712.0,720.0,...,BOTH,ACTIVE,KINGSTON,KINGSTON,KINGSTON,Ground Level,ST,Victoria St,"{""coordinates"": [[-76.50340285566469, 44.24130...","44.2414952481128,-76.50354210177557"
2,4937,NaN,NaN,NaN,615 OFF WB,615 OFF WB,SIR JOHN A. MACDONALD BLVD,401,NaN,NaN,...,UP,ACTIVE,KINGSTON,KINGSTON,KINGSTON,Ground Level,NaN,615 Off Wb,"{""coordinates"": [[-76.51822585972158, 44.27195...","44.272853663834184,-76.51886789241843"
3,3022,EMERALD,ST,NaN,EMERALD ST,CAVENDISH CRES,CAVENDISH CRES,6R,NaN,NaN,...,BOTH,ACTIVE,KINGSTON,KINGSTON,KINGSTON,Ground Level,ST,Emerald St,"{""coordinates"": [[-76.55709444382047, 44.26273...","44.26313792580296,-76.55709493535714"
4,1130,CAITLIN,CRES,NaN,CAITLIN CRES,CAITLIN CRES,CAITLIN CRES,32R,1066.0,1168.0,...,BOTH,ACTIVE,KINGSTON,KINGSTON,KINGSTON,Ground Level,CR,Caitlin Cres,"{""coordinates"": [[-76.60268842897942, 44.25709...","44.25721949771062,-76.60198870361369"


In [24]:
neighbourhood_info = pd.read_csv('neighbourhood-census-profiles-family-housing-mobility.csv')
neighbourhood_info.head()

,Community Name,Area Type,Area ID Number,Total Population 2016,Total Population 2011,Population Change,Population Aged 0-4,Population Aged 5-9,Population Aged 10-14,Population Aged 15-19,...,Non-movers - Mobility status 5 years ago,Movers - Mobility status 5 years ago,Non-Migrant - Mobility status 5 years ago,Migrant - Mobility status 5 years ago,Internal Migrant - Mobility status 5 years ago,Intraprovincial Migrant - Mobility status 5 years ago,Interprovincial Migrant - Mobility status 5 years ago,External Migrant - Mobility status 5 years ago,GeoJSON,geo_point_2d
0,Joyceville / Brewers Mills,Neighbourhood,43,1770,1980,-210,60,90,75,100,...,835,395,235,165,160,150,10,10,"{""coordinates"": [[[-76.24469013039291, 44.4191...","44.38538292067768,-76.28596206890569"
1,Alwington,Neighbourhood,34,1115,1090,25,35,55,55,80,...,545,350,140,215,155,120,30,60,"{""coordinates"": [[[-76.4994731896322, 44.22254...","44.2228671760366,-76.50605465502022"
2,Cataraqui North,Neighbourhood,5,4475,3370,1105,285,260,245,255,...,1980,1960,1140,820,715,595,120,105,"{""coordinates"": [[[-76.53363017973487, 44.2587...","44.26913255386315,-76.55242248231944"
3,Cataraqui River East,Neighbourhood,37,3075,2740,335,115,115,140,130,...,1455,1420,740,680,600,360,240,80,"{""coordinates"": [[[-76.4390714449385, 44.25926...","44.25216048378566,-76.45480098302505"
4,Waterloo Village,Neighbourhood,13,2130,2065,65,110,145,145,125,...,1180,660,410,255,235,195,30,25,"{""coordinates"": [[[-76.55285512283804, 44.2581...","44.25482479014073,-76.55088174548007"


In [36]:
neighbourhood_info.drop([23], inplace = True) # drop kingston row

In [37]:
lat = neighbourhood_info['geo_point_2d'].apply(lambda x: x.split(',')).apply(lambda x: float(x[0]))
lon = neighbourhood_info['geo_point_2d'].apply(lambda x: x.split(',')).apply(lambda x: float(x[1]))
neighbourhood_info['lat'] = lat
neighbourhood_info['lon'] = lon

In [77]:
fig = px.scatter_mapbox(neighbourhood_info, lat="lat", lon="lon", hover_name="Community Name", hover_data=["Total Population 2016"],
                        color ="Total Population 2016", color_continuous_scale=px.colors.cyclical.IceFire, size ="Total Population 2016",  zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.show()



In [125]:
from functools import partial
import pyproj
from shapely.geometry import Polygon
import shapely.ops as ops
from shapely.ops import transform

In [126]:
neighbourhood_info.reset_index(inplace = True)

In [127]:
# region area
area = []
for i in range(neighbourhood_info.shape[0]):
    #print(i)
    l = json.loads(neighbourhood_info['GeoJSON'][i])['coordinates'][0]
    if i == 12 or i == 29 or i == 36:
        l = json.loads(neighbourhood_info['GeoJSON'][i])['coordinates'][0][0]
    polygon = Polygon(l)
    area +=[polygon.area]

In [128]:
area[0:5]

[0.01130255958508533,
 7.138383799617402e-05,
 0.0008859527299821643,
 0.00033429413050095407,
 0.00010202239466511153]

In [129]:
sum_area = sum(area)
for i in range(len(area)):
    area[i] = area[i]/sum_area #percentage of area in kingston
    # kingston is 450.4 km²
    area[i] = area[i] * 450.4

In [130]:
neighbourhood_info['area(km²)'] = area
neighbourhood_info['population/area(km²)'] = neighbourhood_info['Total Population 2016'] / neighbourhood_info['area(km²)'] 
fig = px.scatter_mapbox(neighbourhood_info, lat="lat", lon="lon", hover_name="Community Name", hover_data=['population/area(km²)'],
                        color ='population/area(km²)', color_continuous_scale=px.colors.cyclical.IceFire, size ='population/area(km²)',  zoom=6, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.show()



In [131]:
area

[95.96688289749062,
 0.60610027049024,
 7.522377674797156,
 2.838398279044446,
 0.8662437148013861,
 0.577873193529655,
 1.454187924108111,
 17.709170386674096,
 3.090879700578032,
 1.5668889779603352,
 0.9695076816595145,
 2.1584376495808337,
 2.1393245448233107,
 1.5693846853772475,
 0.7828949697107724,
 1.8571407890578502,
 1.1104044591225477,
 7.071024310405044,
 1.196625618738771,
 6.868056767956546,
 18.45413497991015,
 0.8081705572635208,
 5.856852613051551,
 3.4215184207347527,
 3.78377224305541,
 1.4072095771152513,
 1.2850877995273235,
 46.23457041568843,
 5.178382707019108,
 63.70971039395814,
 1.7469478337684452,
 2.561836804745797,
 85.32083762165074,
 1.5773681098232055,
 2.05009458395547,
 17.519618094051353,
 9.168271079960238,
 5.827545545614824,
 15.98818110099855,
 0.5780850222012504]

In [68]:
neighbourhood_info.columns.tolist()

['Community Name',
 'Area Type',
 'Area ID Number',
 'Total Population 2016',
 'Total Population 2011',
 'Population Change',
 'Population Aged 0-4',
 'Population Aged 5-9',
 'Population Aged 10-14',
 'Population Aged 15-19',
 'Population Aged 20-24',
 'Population Aged 25-29',
 'Population Aged 30-34',
 'Population Aged 35-39',
 'Population Aged 40-44',
 'Population Aged 45-50',
 'Population Aged 50-54',
 'Population Aged 55-59',
 'Population Aged 60-64',
 'Population Aged 65-69',
 'Population Aged 70-74',
 'Population Aged 75-79',
 'Population Aged 80+',
 'Population Median Age',
 'Total - Marital status',
 'Married or living common law - Marital status',
 'Never married - Marital status',
 'Separated/ Divorced - Marital status',
 'Widowed - Marital status',
 'Total - Number of census families in private households',
 'Couples without children in their census family - Number of census families in private households',
 'Couples with children in their census family - Number of census fa

In [70]:
fig = px.scatter_mapbox(neighbourhood_info, lat="lat", lon="lon", hover_name="Community Name", hover_data=["Total Population 2016"],
                        color_discrete_sequence=["fuchsia"], size ="Renter - Private households by tenure",  zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.show()


In [72]:
neighborhood_other_info = pd.read_csv('neighbourhood-census-profiles-income-occupation-education.csv')
neighborhood_other_info.head()

,Community Name,Area Type,Area ID Number,Median after-tax income of households in 2015 (dollars),Prevalence of low income in 2015 based on after-tax low-income measure (percent),Total - Household after-tax income groups in 2015 for private households (dollars),"0-9,999 - Household after-tax income groups in 2015 for private households (dollars)","10,000-19,999 - Household after-tax income groups in 2015 for private households (dollars)","20,000-29,000 - Household after-tax income groups in 2015 for private households (dollars)","30,000-39,000 - Household after-tax income groups in 2015 for private households (dollars)",...,"Personal, protective and transportation services - Major field of study",Personal and culinary services - Major field of study,"Military science, leadership and operational art - Major field of study",Military technologies and applied sciences - Major field of study,Security and protective services - Major field of study,Transportation and materials moving - Major field of study,Other - Major field of study,"Multidisciplinary/interdisciplinary studies, other - Major field of study",GeoJSON,geo_point_2d
0,Kingston,Neighbourhood,0,59541,14.8,53515,1725,3980,5355,5265,...,4130,2175,170,155,1310,315,15,10,"{""coordinates"": [[[-76.60112583326217, 44.1754...","44.301669203165794,-76.46058510382774"
1,Bayridge West,Neighbourhood,10,71427,7.4,1175,25,30,60,115,...,100,55,0,0,35,10,0,0,"{""coordinates"": [[[-76.59061344049432, 44.2426...","44.24610543479501,-76.59993818176103"
2,Greenwood / St Lawrence South,Neighbourhood,40,103345,2.6,1870,30,0,45,50,...,260,70,60,30,90,10,10,0,"{""coordinates"": [[[-76.3871960849859, 44.28804...","44.27422750914604,-76.41678636546013"
3,Lemoine Point,Neighbourhood,14,85897,7.3,525,10,10,35,40,...,55,20,0,0,40,0,0,0,"{""coordinates"": [[[-76.59318756066864, 44.2388...","44.22699552164968,-76.60595048418064"
4,Strathcona Park,Neighbourhood,19,62439,10.4,1335,10,50,105,160,...,100,60,0,0,30,0,0,0,"{""coordinates"": [[[-76.51871079433725, 44.2450...","44.253291287235896,-76.526377201127"


In [76]:
lat = neighborhood_other_info['geo_point_2d'].apply(lambda x: x.split(',')).apply(lambda x: float(x[0]))
lon = neighborhood_other_info['geo_point_2d'].apply(lambda x: x.split(',')).apply(lambda x: float(x[1]))
neighborhood_other_info['lat'] = lat
neighborhood_other_info['lon'] = lon
fig = px.scatter_mapbox(neighborhood_other_info, lat="lat", lon="lon", hover_name="Community Name", hover_data=["Median after-tax income of households in 2015 (dollars)"],
                        color ="Median after-tax income of households in 2015 (dollars)", color_continuous_scale=px.colors.cyclical.IceFire, size ="Median after-tax income of households in 2015 (dollars)",  zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.show()
